# Load DNS and mDNS datasets

In [7]:
import os
import sys

import numpy as np
import pandas as pd

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
import src.temporal_loader_v2 as tl
from src.utils import to_homogeneous


In [8]:
cuda_device = 4

if torch.cuda.is_available():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    torch.cuda.set_device(cuda_device)
    
torch.manual_seed(42)

## Load Graphs

In [9]:
kg_path = lambda graph_name: f'../data/{graph_name}'
dataset = tl.DNS(root=kg_path('DNS_2m'), start=0, end=6, test_list=[7], balance_gt=True, domain_file='domains2.csv')

Total labeled 897635
Labeled node count for 0, 6: 31778
After balancing labeled count: 31282
Labeled node count for 0, 7: 2610


#### DNS

In [10]:
data = dataset.train_data # training data
# data = to_homogeneous(dataset.train_data) # training data
# test_data = to_homogeneous(dataset.test_data[0])
dataset.train_data.metadata()

(['domain_node', 'ip_node'],
 [('domain_node', 'apex', 'domain_node'),
  ('domain_node', 'resolves', 'ip_node'),
  ('domain_node', 'similar', 'domain_node')])

#### GNN

In [11]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import SAGEConv, HeteroConv, Linear


class HeteroGNN(torch.nn.Module):
    def __init__(self, metadata, hidden_channels, out_channels, num_layers):
        super().__init__()

        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HeteroConv({
                edge_type: SAGEConv((-1,-1), hidden_channels)
                for edge_type in metadata[1]
            })
            self.convs.append(conv)

        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict):
        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)
            x_dict = {key: F.leaky_relu(x) for key, x in x_dict.items()}
        return self.lin(x_dict['domain_node'])
    



In [12]:
cuda_device = 7
torch.manual_seed(42)
from src.utils import score

def train(model, data, optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(data.x_dict, data.edge_index_dict)
    mask = data['domain_node'].train_mask
    loss = F.cross_entropy(out[mask], data['domain_node'].y[mask])
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test(model, data):
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict).argmax(dim=-1)

    accs = []
    for split in ['train_mask', 'val_mask']:
        mask = data['domain_node'][split]
        acc = (pred[mask] == data['domain_node'].y[mask]).sum() / mask.sum()
        accs.append(float(acc))
    return accs

def experiment(model,start,end,test_list, model_type):
    kg_path = lambda graph_name: f'../data/{graph_name}'

    dataset = tl.DNS(root=kg_path('DNS_2m'), start=start, end=end, test_list=test_list, balance_gt=False, domain_file='domains2.csv')
    # data = to_homogeneous(dataset.train_data) # training data
    data = dataset.train_data # training data

    if torch.cuda.is_available():
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        torch.cuda.set_device(cuda_device)

        data, model = data.to(device), model.to(device)

    with torch.no_grad():  # Initialize lazy modules.
        out = model(data.x_dict, data.edge_index_dict)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)

    for epoch in range(0, 201):
        loss = train(model, data, optimizer)
        train_acc, val_acc = test(model,data)
        if epoch % 20 == 0:
            print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, '
                f'Val: {val_acc:.4f}')
        
    model.eval()
    for index, test_data in enumerate(dataset.test_data):
        # test_data = to_homogeneous(test_data)
        test_data = test_data.to(device)
        with torch.no_grad():
            pred = model(test_data.x_dict, test_data.edge_index_dict).argmax(dim=-1)
        mask = test_data['domain_node']['val_mask']
        scores = score(pred[mask],test_data['domain_node'].y[mask])
        with open("../results_copy.csv", "a") as logger:
            logger.write("{},{},{},{},".format(model_type,start,end,index))
            logger.write(",".join(str(x) for x in scores.values()))
            logger.write('\n')


        for metric, val in scores.items():
            print(metric, ':{:.4f}'.format(val))
    
for model_type in ['heterosage']:
    for i in range(5):
        # model_type='gcn'  
        # data.x.size(1) 
        model = HeteroGNN(data.metadata(), hidden_channels=64, out_channels=2,
                  num_layers=2)
        experiment(model,i,i+6,[i+7,i+8], model_type)

Total labeled 897635
Labeled node count for 0, 6: 31778
Labeled node count for 0, 7: 2610
Labeled node count for 0, 8: 2083
Epoch: 000, Loss: 0.6967, Train: 0.7126, Val: 0.7262
Epoch: 020, Loss: 0.4232, Train: 0.8100, Val: 0.8107
Epoch: 040, Loss: 0.3776, Train: 0.8194, Val: 0.8197
Epoch: 060, Loss: 0.3736, Train: 0.8207, Val: 0.8214
Epoch: 080, Loss: 0.3774, Train: 0.8215, Val: 0.8238
Epoch: 100, Loss: 0.3713, Train: 0.8219, Val: 0.8225
Epoch: 120, Loss: 0.3703, Train: 0.8216, Val: 0.8230
Epoch: 140, Loss: 0.3705, Train: 0.8217, Val: 0.8217
Epoch: 160, Loss: 0.3700, Train: 0.8221, Val: 0.8228
Epoch: 180, Loss: 0.3703, Train: 0.8216, Val: 0.8216
Epoch: 200, Loss: 0.3689, Train: 0.8222, Val: 0.8227
tn, fp, fn, tp 1361 244 154 851
acc :0.8475
f1 :0.8486
auc :0.8474
prec :0.7772
recall :0.8468
fpr :0.1520
mi_f1 :0.8475
ma_f1 :0.8415
tn, fp, fn, tp 1111 198 133 641
acc :0.8411
f1 :0.8423
auc :0.8385
prec :0.7640
recall :0.8282
fpr :0.1513
mi_f1 :0.8411
ma_f1 :0.8326
Total labeled 897635
La